In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/content/sentiment_analysis_nepali_final.csv')

In [3]:
data.head()

,Unnamed: 0,Sentences,Sentiment
0,0,"म एक शिक्षक , शिक्षा क्षेत्रमा रमाएको मान्छे ।...",1
1,1,म सरकारी स्कूल/कलेजमा पढेर करीब १२ बर्ष भन्दा ...,1
2,2,कति राम्रो शिव मन्दिर देख्न पाइयो कुन ठाउको हो...,1
3,3,मारुनी भन्ने वितिकै सामान्य नाचनीमा आधारित कथा...,1
4,4,यो फ्लिम हेरिसकेपछी थाहा भयो कि किन दर्सकहरुले...,1


In [4]:
data.sample(5)

,Unnamed: 0,Sentences,Sentiment
22081,19761,इपिडिमियोलोजी तथा रोग नियन्त्रण महाशाखा र नेपा...,1
25638,23318,हामी सवैलाई खुशिको खवर बर्षिए बाग्लुङकी आमाले ...,1
17585,15257,कर्णाली प्रदेशमा कोभिड बाट संक्रमित हुनेको कुल...,-1
20929,18609,शिक्षण अस्पतालले भन्यो अस्पतालकर्मीमा कोभिड सङ...,1
34388,32071,कोभिड को यत्रो जोखिम र त्रास हुॅदा प्रदेश र स्...,-1


In [5]:
df = data.drop("Unnamed: 0", axis=1)

In [6]:
df = df.rename(columns={'Sentences': 'text','Sentiment':'labels'})


In [7]:
df.sample(5)

,text,labels
25327,सम्पुर्ण जनता लेखिएको देखिएको बोलिएको सुनिएको ...,1
8697,कोरोना भाइरस कोभिड को विश्वव्यापी महामारीका का...,-1
14941,भारतबाट आएको कोभिड लाई समर्थन गर्दै प्रचण्डपथम...,0
19639,नेपाल उद्योग परिसंघले आयोजना गरेको वेबिनारमा क...,0
29360,चीनमा कोभिड सङ्क्रमणबाट मृत्यु भएका नागरिकको स...,-1


In [8]:
df['text'].isnull().sum()

0

In [9]:
df['labels'].isnull().sum()

0

In [10]:
df = df.dropna(subset=['labels'])

In [11]:
df['labels'].isnull().sum()

0

In [12]:
from nltk.corpus import stopwords

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
stopword=set(stopwords.words('nepali'))
X_data=[]
for i in df['text']:
    text=' '.join([word for word in i.split() if word not in stopword])
    X_data.append(text.strip())

In [15]:
Y=list(df['labels'])

In [16]:
from sklearn.model_selection import train_test_split


In [17]:
x_train,x_test,y_train,y_test=train_test_split(X_data,Y,test_size=0.25,random_state=0)

In [18]:
len(x_train)

26841

In [19]:
len(x_test)

8948

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
def tfidf_features(X_train, X_test):
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    return X_train_tfidf,X_test_tfidf,tfidf_vectorizer


In [22]:
X_train_tfidf, X_test_tfidf, tfidf_vectorizer = tfidf_features(x_train, x_test)

In [23]:
from sklearn import svm

In [24]:
# cs = [0.01, 0.1, 1, 5, 10]
# for c in cs:
#  model=svm.SVC(C=c,kernel='poly',gamma='auto')
#  model.fit(X_train_tfidf,y_train)
#  print('Testing Accuracy for C=%s :%s'%(c,model.score(X_test_tfidf, y_test)))

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'C': [0.01, 0.1, 1, 10, 100],
              'kernel': ['rbf'], #, 'poly', 'linear'
              'gamma': ['scale', 'auto']}

grid_search = GridSearchCV(svm.SVC(), parameters, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Test set score: {:.2f}".format(grid_search.score(X_test_tfidf, y_test)))


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [ ]:
# model=svm.SVC(C=10,kernel='poly',gamma='scale')
# model.fit(X_train_tfidf,y_train)
# print('Testing Accuracy for C=%s :%s'%(c,model.score(X_test_tfidf, y_test)))

In [ ]:
nepali_text=[["हामिले सहयोग गर्नुपर्छ"],["सलाम छ रबि सर लाई"],["के खानु चाही अबैध रुपमा बिदेश जानु पर्यो"]]
for text in nepali_text:
    pred=tfidf_vectorizer.transform(text)
    prediction = np.squeeze(model.predict(pred))
    if prediction == 1:
        print("%s" % (np.squeeze(text)), "Positive")
    elif prediction == -1:
        print("%s" % (np.squeeze(text)), "Negative")
    else:
        print("%s" % (np.squeeze(text)), "Neutral")